In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pandas_profiling import ProfileReport
from matplotlib_venn import venn2
from matplotlib.pyplot import figure

%run utils.ipynb
%run predict.ipynb

figure(num=None, figsize=(20, 16), dpi=80, facecolor='w', edgecolor='k')

df = pd.read_csv('./data/train.csv')

# First we perform an explorative data analysis
# profile = ProfileReport(df, title='Titanic raw data analysis', explorative=True)
# profile.to_widgets() # gross overview
# profile.to_file('./analysis/pandas-analysis.html')


# Clean data
df = clean_data(df)

# sns.catplot('Survived', 'Age', kind='point', data=df)
# plt.show()


# Make all values numeric
# Sex (0 => male, 1 => female)
# Title (0 => Mr, 1 => Miss, 2 => Mrs)
# Age (    
    # 0 => infant <= 2
    # 1 => toddler <= 4
    # 2 => child <= 12
    # 3 => teen <= 18
    # 4 => young adult <= 25
    # 5 => adult <= 35
    # 6 => senior <= 55
    # 7 => old 35 - 55
    # 8 => elderly 55 - 150
# )
# Fare Group (0 => <10$, 1 => <10, 2 => <20, 3 => <50, 4 => 5000)
df = numerize_data(df)

# profile = ProfileReport(df, title='Titanic raw data analysis', explorative=True)
# profile.to_file('./analysis/pandas-analysis4.html')
# print(df)
autoMlPredict(df)

# print(df)


Create directory AutoML_1
AutoML task to be solved: binary_classification
AutoML will use algorithms: ['CatBoost']
AutoML will optimize for metric: logloss
1_Default_CatBoost final logloss 0.0003253282345639571 time 10.32 seconds
    Predicted  Survived  Score
0           1         1    1.0
1           0         0    1.0
2           0         0    1.0
3           0         0    1.0
4           0         0    1.0
..        ...       ...    ...
85          0         0    1.0
86          1         1    1.0
87          0         0    1.0
88          1         1    1.0
89          0         0    1.0

[90 rows x 3 columns]
Test MSE: 0.0


1.0